# Introduction

This notebook answers the following question:  

- For those who haven’t donated, what is the probability of a gift in the following year of their first instance of standard volunteerism?  

by Fred Etter - December, 2019

In [1]:
# Import modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import sklearn
from sklearn.feature_selection import SelectFromModel
from datetime import datetime
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import seaborn as sns

In [2]:
# Read in the data
df = pd.read_csv('file1_12_3.csv', low_memory=False)

In [3]:
# show the first 5 lines of the data
df.head()

,ContactId,Year,PledgeTotal,VolType,VolunteerActivityCnt,BirthYear,Gender
0,874ddbce-11cd-e111-941f-00259073dc22,2007,52.00,NaN,NaN,1977.0,female
1,9a4ddbce-11cd-e111-941f-00259073dc22,2007,80.00,NaN,NaN,1968.0,female
2,5e4edbce-11cd-e111-941f-00259073dc22,2007,10000.00,NaN,NaN,1958.0,male
3,c150dbce-11cd-e111-941f-00259073dc22,2007,120.00,NaN,NaN,NaN,male
4,0b53dbce-11cd-e111-941f-00259073dc22,2007,500.16,NaN,NaN,1947.0,female


In [4]:
# show the number of rows and columns of the original data
df.shape

(385722, 7)

In [5]:
# drop Council, Committe, Board members
df = df.drop(df[df.VolType == 'Council, Committe or Board'].index).fillna(0)

In [6]:
# create a dataframe that only has volunteers of 'Standard' type:
df_gender = df.copy()
df_gender = df_gender.fillna(0)
df_gender = df_gender.drop(df_gender[df_gender.VolType == 0].index).fillna(0)
df_std_only = df_gender.drop(df_gender[df_gender.VolType == 'Council, Committe or Board'].index).fillna(0)

In [7]:
# create a series that captures the earliest year that someone volunteered
a = df_std_only.groupby('ContactId')['Year'].transform('min')

In [8]:
# create a new column and populate it:
df_std_only['min_year'] = a

In [9]:
# display a random sample of 10 rows of the dataframe
df_std_only.sample(10)

,ContactId,Year,PledgeTotal,VolType,VolunteerActivityCnt,BirthYear,Gender,min_year
381064,c14cb1d1-11cd-e111-941f-00259073dc22,2011,0.00,Standard,1.0,1977.0,male,2011
375844,e4d52ba3-a0dd-e211-a0e0-4040184c1c1a,2013,0.00,Standard,1.0,0.0,female,2013
378825,c9b699cf-11cd-e111-941f-00259073dc22,2010,0.00,Standard,1.0,1966.0,female,2010
380692,344015d2-11cd-e111-941f-00259073dc22,2012,0.00,Standard,1.0,0.0,male,2012
382008,425ab1d1-11cd-e111-941f-00259073dc22,2012,0.00,Standard,1.0,0.0,female,2011
378341,573f15d2-11cd-e111-941f-00259073dc22,2011,0.00,Standard,1.0,0.0,female,2011
385240,c1c762d1-11cd-e111-941f-00259073dc22,2015,1084.86,Standard,4.0,0.0,female,2010
382906,823e15d2-11cd-e111-941f-00259073dc22,2013,0.00,Standard,1.0,1983.0,female,2011
377332,a751b7d0-11cd-e111-941f-00259073dc22,2013,265.00,Standard,1.0,0.0,male,2013
379756,30e777a0-a231-e711-b10c-005056975e92,2017,0.00,Standard,1.0,0.0,female,2017


In [10]:
# drop rows where Year = min_year
df_std_only_t = df_std_only.drop(df_std_only[df_std_only.Year != df_std_only.min_year].index)

In [11]:
# drop rows where there was no pledge
df_init = df_std_only_t.drop(df_std_only_t[df_std_only_t.PledgeTotal != 0].index)

In [12]:
# number of rows and columns of the dataframe
df_init.shape

(4089, 8)

In [13]:
# show a random sample of 10 rows of the dataframe
df_init.sample(10)

,ContactId,Year,PledgeTotal,VolType,VolunteerActivityCnt,BirthYear,Gender,min_year
377952,a914fde4-6579-e811-80ea-0050569e2a4f,2018,0.0,Standard,1.0,0.0,female,2018
378075,910ef4d6-59c9-e211-a0e0-4040184c1c1a,2012,0.0,Standard,1.0,0.0,female,2012
382742,7893efd3-302d-e311-a76b-4040184c1c1a,2013,0.0,Standard,1.0,0.0,female,2013
380950,f482d7ee-3ac3-e811-8101-0050569e2a4f,2018,0.0,Standard,1.0,0.0,0,2018
380370,932e8f61-1907-e611-828d-26d4160798d6,2016,0.0,Standard,1.0,0.0,0,2016
377982,a343e8cf-11cd-e111-941f-00259073dc22,2016,0.0,Standard,1.0,1947.0,female,2016
378308,784cb1d1-11cd-e111-941f-00259073dc22,2010,0.0,Standard,1.0,0.0,female,2010
375818,ba19f980-dc7c-4d7b-8588-750e24506578,2019,0.0,Standard,1.0,0.0,male,2019
380922,953db1d1-11cd-e111-941f-00259073dc22,2011,0.0,Standard,1.0,1970.0,female,2011
376120,0c7355db-23c2-e211-a0e0-4040184c1c1a,2012,0.0,Standard,1.0,0.0,male,2012


The above dataframe shows the volunteers who volunteered and have never donated.

In [14]:
# create a new column that add 1 year to the 'min_year' column
df_init['y2'] = df_init.min_year + 1

In [15]:
# display the first 5 rows of the dataframe
df_init.head()

,ContactId,Year,PledgeTotal,VolType,VolunteerActivityCnt,BirthYear,Gender,min_year,y2
375651,4c58f4ca-3545-e511-92f8-26d4160798d6,2015,0.0,Standard,1.0,0.0,0,2015,2016
375652,e34cb1d1-11cd-e111-941f-00259073dc22,2011,0.0,Standard,1.0,0.0,female,2011,2012
375653,29f9f78a-6681-e411-9b7f-26d4160798d6,2018,0.0,Standard,1.0,1955.0,female,2018,2019
375655,34aaf5d1-d3d1-e211-a0e0-4040184c1c1a,2012,0.0,Standard,1.0,0.0,female,2012,2013
375658,9350b1d1-11cd-e111-941f-00259073dc22,2011,0.0,Standard,1.0,0.0,female,2011,2012


In [16]:
# create a new dataframe to capture the pledge amount for the year after they volunteered
new_df = pd.merge(df_init, df, how='left', left_on=['ContactId','y2'], right_on = ['ContactId', 'Year'])

In [17]:
# display the number of rows and columns of dataframe
new_df.shape

(4089, 15)

In [18]:
# view a random sample of 5 rows
new_df.sample(5)

,ContactId,Year_x,PledgeTotal_x,VolType_x,VolunteerActivityCnt_x,BirthYear_x,Gender_x,min_year,y2,Year_y,PledgeTotal_y,VolType_y,VolunteerActivityCnt_y,BirthYear_y,Gender_y
491,4e52b1d1-11cd-e111-941f-00259073dc22,2014,0.0,Standard,1.0,1953.0,female,2014,2015,NaN,NaN,NaN,NaN,NaN,NaN
309,403eb1d1-11cd-e111-941f-00259073dc22,2012,0.0,Standard,1.0,0.0,female,2012,2013,NaN,NaN,NaN,NaN,NaN,NaN
2331,d70675ce-11cd-e111-941f-00259073dc22,2013,0.0,Standard,1.0,0.0,female,2013,2014,2014.0,496.08,Standard,1.0,0.0,female
2397,d38a7601-1407-e611-828d-26d4160798d6,2016,0.0,Standard,1.0,0.0,0,2016,2017,NaN,NaN,NaN,NaN,NaN,NaN
3683,a44de8cf-11cd-e111-941f-00259073dc22,2013,0.0,Standard,1.0,0.0,male,2013,2014,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# fill NaNs with 0
new_df = new_df.fillna(0)

In [20]:
# drop all rows with 0 in pledge total
new_df = new_df.drop(new_df[new_df.PledgeTotal_y == 0].index)

In [21]:
# new number of rows which is the number of people who donated the year after they volunteered
new_df.shape

(255, 15)

In [22]:
# capture the mean donation
new_df.PledgeTotal_y.mean()

237.19337254901964

In [23]:
# get the probability of those who donated the year after they volunteered
255/4089

0.06236243580337491

So, **6.2%** of volunteers who had never donated, donated the following year. In other words, there is a **6.2%** chance that a volunteer who has never donated will make a donation in the year following there first volunteer activity.

There average donation for this group was **237** USD.

### Now, check about the case for 5 years later:

In [24]:
##### All of the following steps are same as above except adding 5 years instead of 1 #######
df_init['y5'] = df_init.min_year + 5

In [25]:
new_df_5 = pd.merge(df_init, df, how='left', left_on=['ContactId','y5'], right_on = ['ContactId', 'Year'])

In [26]:
new_df_5.head()

,ContactId,Year_x,PledgeTotal_x,VolType_x,VolunteerActivityCnt_x,BirthYear_x,Gender_x,min_year,y2,y5,Year_y,PledgeTotal_y,VolType_y,VolunteerActivityCnt_y,BirthYear_y,Gender_y
0,4c58f4ca-3545-e511-92f8-26d4160798d6,2015,0.0,Standard,1.0,0.0,0,2015,2016,2020,NaN,NaN,NaN,NaN,NaN,NaN
1,e34cb1d1-11cd-e111-941f-00259073dc22,2011,0.0,Standard,1.0,0.0,female,2011,2012,2016,NaN,NaN,NaN,NaN,NaN,NaN
2,29f9f78a-6681-e411-9b7f-26d4160798d6,2018,0.0,Standard,1.0,1955.0,female,2018,2019,2023,NaN,NaN,NaN,NaN,NaN,NaN
3,34aaf5d1-d3d1-e211-a0e0-4040184c1c1a,2012,0.0,Standard,1.0,0.0,female,2012,2013,2017,NaN,NaN,NaN,NaN,NaN,NaN
4,9350b1d1-11cd-e111-941f-00259073dc22,2011,0.0,Standard,1.0,0.0,female,2011,2012,2016,2016.0,54.6,0,0.0,0.0,female


In [27]:
new_df_5.shape

(4089, 16)

In [28]:
new_df_5 = new_df_5.fillna(0)

In [29]:
new_df_5 = new_df_5.drop(new_df_5[new_df_5.PledgeTotal_y == 0].index)

In [30]:
new_df_5.shape

(192, 16)

In [31]:
new_df_5.PledgeTotal_y.mean()

267.56671875

In [32]:
192/4097

0.046863558701488894

# Conclusion:  

After the first year of volunteering, those volunteers gave a gift **6.2%** of the time the following year and **4.7%** of the time in the 5th year after volunteering.  

The average gift 1 year later was **237 USD** and it was **268 USD** 5 years later.